In [ ]:
import requests

def get_geospatial_whole_island(api_url, api_key, layer_id):
    """
    Function to fetch data from the LTA DataMall API for geospatial whole island.

    Use cases: 
    - 2.22 Geospatial Whole Island
    
    Parameters:
    - api_url (str): The API endpoint URL for geospatial data.
    - api_key (str): Your LTA DataMall API key.
    - layer_id (str): Name of the geospatial layer (case sensitive).
    
    Returns:
    - str: A link to download the geospatial dataset.
    """
    headers = {
        'AccountKey': api_key,
        'accept': 'application/json'
    }

    params = {'ID': layer_id}
    
    response = requests.get(api_url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if 'value' in data:
            download_link = data['value'][0]['Link']  # Adjust index if there are multiple links
            return download_link
        else:
            print("No 'value' key found in the response.")
            return None
    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return None

In [ ]:
# Sample call
# geospatial_link = get_geospatial_whole_island(api_url, api_key, layer_id="TrainStation")
# print(geospatial_link)